<a href="https://colab.research.google.com/github/ale-camer/Finance/blob/master/Volatility_Surface_Implicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yahooquery 

In [35]:
def vol_surf_imp(ticker = 'aapl'):
        
    """ Provide 3D plot of the volatilty surface of calls and puts of any stock listed in the New York Stock Exchange (NYSE).
    The whole data is download and calculated by Yahoo Finance.
    Only is considered options that were traded in the last business day. In case of holidays, no graph will be displayed. """

    from datetime import datetime, timedelta
    from yahooquery import Ticker
    import pandas as pd
    import plotly.graph_objects as go
    import warnings
    warnings.filterwarnings('ignore')
    
    tick_ = ticker    
    df = Ticker(tick_, asynchronous=True).option_chain.rename_axis().reset_index() # download data
    test_date = datetime.today()
    diff = 1
    if test_date.weekday() == 0:
        diff = 3
    elif test_date.weekday() == 6:
        diff = 2
    else :
        diff = 1
    res = test_date - timedelta(days=diff)
    df = df[df['lastTradeDate'].dt.strftime('%d-%m-%Y') == res.strftime('%d-%m-%Y')] # subset data from last business day

    calls = df[df['optionType'] == 'calls'] # calls data
    calls = pd.pivot_table(data = calls, 
                        values = 'impliedVolatility',
                        index = 'strike',
                        columns = 'expiration')
    a = calls.isna().sum(axis = 1) / calls.shape[1]
    b = a[a < 0.5].index
    calls = calls[calls.index.isin(b)]
    calls = calls.fillna(method = 'bfill', axis = 0).fillna(method = 'ffill', axis = 0)   
    calls = calls.fillna(method = 'bfill', axis = 1).fillna(method = 'ffill', axis = 1)   
    
    puts = df[df['optionType'] == 'puts'] # puts data
    puts = pd.pivot_table(data = puts, 
                        values = 'impliedVolatility',
                        index = 'strike',
                        columns = 'expiration')
    a = puts.isna().sum(axis = 1) / puts.shape[1]
    b = a[a < 0.5].index
    puts = puts[puts.index.isin(b)]
    puts = puts.fillna(method = 'bfill', axis = 0).fillna(method = 'ffill', axis = 0)   
    puts = puts.fillna(method = 'bfill', axis = 1).fillna(method = 'ffill', axis = 1)   
    
    xc, yc, zc = calls.columns, calls.index.values, calls.values
    xp, yp, zp = puts.columns, puts.index.values, puts.values
    
    layout = go.Layout(xaxis = go.layout.XAxis(
        title = go.layout.xaxis.Title(
            text = 'Date')),
        yaxis = go.layout.YAxis(
            title = go.layout.yaxis.Title(
                text = 'Strike')))
    # graphs
    fig = go.Figure(data = [go.Surface(z = zc, y = yc, x = xc)], layout = layout)
    fig.update_layout(title = 'Call Implied Volatility Surface',
                      scene = dict(
                        xaxis_title = 'Date',
                        yaxis_title = 'Strike',
                        zaxis_title = 'Implied Volatility'),
                      autosize = False,
                      width = 800, height = 800,
                      margin = dict(l = 65, r = 50, b = 65, t = 90))
    fig.show()
    
    fig = go.Figure(data= [go.Surface(z = zp, y = yp, x = xp)], layout = layout)
    fig.update_layout(title = 'Put Implied Volatility Surface',
                      scene = dict(
                        xaxis_title = 'Date',
                        yaxis_title = 'Strike',
                        zaxis_title = 'Implied Volatility'),
                      autosize = False,
                      width = 800, height = 800,
                      margin = dict(l = 65, r = 50, b = 65, t = 90))
    fig.show()

In [36]:
vol_surf_imp()    

In [37]:
vol_surf_imp('meli')    

In [38]:
vol_surf_imp('msft')    

In [39]:
vol_surf_imp('fb')    

In [40]:
vol_surf_imp('goog')    